PARTE I

Machine learning para clasificar

Python es el lenguaje preferido para la ciencia de datos debido a NumPy,
Pandas y matplotlib, que son herramientas que facilitan el trabajo con
matrices y dibujan gráficos y pueden trabajar con grandes matrices de datos
de manera eficiente. Pero Spark está diseñado para trabajar con una enorme
cantidad de datos, distribuidos en un clúster.

Datos del paciente del corazón

Encontrará adjunto la base de datos.

Las columnas son:

1. Años
2. Sexo
3. Tipo de dolor torácico (4 valores)
4. Presión arterial en reposo
5. Colesterol sérico en mg/dl
6. Azúcar en sangre en ayunas > 120 mg/dl
7. Resultados electrocardiográficos en reposo (valores 0,1,2)
8. Frecuencia cardíaca máxima alcanzada
9. Angina inducida por el ejercicio
10. Oldpeak = depresión del ST inducida por el ejercicio en relación con el reposo
11. Pendiente del segmento ST de ejercicio máximo
12. Número de vasos principales (0-3) coloreados por fluoroscopia
13. Thal: 3 = normal; 6 = defecto fijo; 7 = defecto reversible

El campo que indica si el paciente tiene un problema cardíaco. Los números
son los siguientes:

Un valor de 3 significa que el paciente está sano (normal). Un valor de 6
significa que se ha solucionado el problema de salud del paciente. Un valor de
7 significa que se puede solucionar el problema de salud del paciente.

Entonces, escriba esta función ENFERMO() para marcar 0 como negativo y 1
como positivo, porque la regresión logística binaria requiere uno de dos
resultados.

También debe crear el dataframe de Spark raw_data usando la operación
transform() y seleccionando solo la columna de características.

In [43]:
#Importar
import pandas as pd
#Dividir en punto y coma ademas de considerar el punto como decimal. Y mostar. 
dataframe = pd.read_csv("heart.csv", sep=';')
display(dataframe)

,años,sexo,dolor,pres,col,sug,electro,fcarmax,angina,oldpeak,pendst,vasprin,thal,enfermo
0,70.0,1.0,4.0,130.0,322.0,0.0,2.0,109.0,0.0,2.4,2.0,3.0,3.0,1
1,67.0,0.0,3.0,115.0,564.0,0.0,2.0,160.0,0.0,1.6,2.0,0.0,7.0,0
2,57.0,1.0,2.0,124.0,261.0,0.0,0.0,141.0,0.0,0.3,1.0,0.0,7.0,1
3,64.0,1.0,4.0,128.0,263.0,0.0,0.0,105.0,1.0,0.2,2.0,1.0,7.0,0
4,74.0,0.0,2.0,120.0,269.0,0.0,2.0,121.0,1.0,0.2,1.0,1.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,52.0,1.0,3.0,172.0,199.0,1.0,0.0,162.0,0.0,0.5,1.0,0.0,7.0,0
266,44.0,1.0,2.0,120.0,263.0,0.0,0.0,173.0,0.0,0.0,1.0,0.0,7.0,0
267,56.0,0.0,2.0,140.0,294.0,0.0,2.0,153.0,0.0,1.3,2.0,0.0,3.0,0
268,57.0,1.0,4.0,140.0,192.0,0.0,0.0,148.0,0.0,0.4,2.0,0.0,6.0,0


In [62]:
data = pd.read_csv("heart.csv", sep=';')
data

,años,sexo,dolor,pres,col,sug,electro,fcarmax,angina,oldpeak,pendst,vasprin,thal,enfermo
0,70.0,1.0,4.0,130.0,322.0,0.0,2.0,109.0,0.0,2.4,2.0,3.0,3.0,1
1,67.0,0.0,3.0,115.0,564.0,0.0,2.0,160.0,0.0,1.6,2.0,0.0,7.0,0
2,57.0,1.0,2.0,124.0,261.0,0.0,0.0,141.0,0.0,0.3,1.0,0.0,7.0,1
3,64.0,1.0,4.0,128.0,263.0,0.0,0.0,105.0,1.0,0.2,2.0,1.0,7.0,0
4,74.0,0.0,2.0,120.0,269.0,0.0,2.0,121.0,1.0,0.2,1.0,1.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,52.0,1.0,3.0,172.0,199.0,1.0,0.0,162.0,0.0,0.5,1.0,0.0,7.0,0
266,44.0,1.0,2.0,120.0,263.0,0.0,0.0,173.0,0.0,0.0,1.0,0.0,7.0,0
267,56.0,0.0,2.0,140.0,294.0,0.0,2.0,153.0,0.0,1.3,2.0,0.0,3.0,0
268,57.0,1.0,4.0,140.0,192.0,0.0,0.0,148.0,0.0,0.4,2.0,0.0,6.0,0


Continuando con los datos del Corazón.

1. Use Standard Scaler para poner todos los números en la misma escala. Esto toma la observación y resta la media, y luego la divide por la desviación estándar.

In [63]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

print(scaler.fit(data))

StandardScaler()


In [64]:
StandardScaler()
print(scaler.mean_)

[5.44333333e+01 6.77777778e-01 3.17407407e+00 1.31344444e+02
 2.49659259e+02 1.48148148e-01 1.02222222e+00 1.49677778e+02
 3.29629630e-01 1.05000000e+00 1.58518519e+00 6.70370370e-01
 4.69629630e+00 4.44444444e-01]


2. Imprima el estado actual de sus datos

In [65]:
#print(scaler.transform(data))
df1 = scaler.fit_transform(data)
df1

array([[ 1.71209356,  0.6894997 ,  0.87092765, ...,  2.47268219,
        -0.87570581,  1.11803399],
       [ 1.38213977, -1.45032695, -0.18355874, ..., -0.71153494,
         1.18927733, -0.89442719],
       [ 0.2822938 ,  0.6894997 , -1.23804513, ..., -0.71153494,
         1.18927733,  1.11803399],
       ...,
       [ 0.1723092 , -1.45032695, -1.23804513, ..., -0.71153494,
        -0.87570581, -0.89442719],
       [ 0.2822938 ,  0.6894997 ,  0.87092765, ..., -0.71153494,
         0.67303154, -0.89442719],
       [ 1.38213977,  0.6894997 ,  0.87092765, ...,  2.47268219,
        -0.87570581,  1.11803399]])

3. Divida los datos en conjuntos de datos de entrenamiento y prueba. Use 50/50.

In [40]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

In [58]:
X = np.array(dataframe.drop(['enfermo'],1))
y = np.array(dataframe['enfermo'])

X

C:\Users\Erik\AppData\Local\Temp\ipykernel_18552\1840235236.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = np.array(dataframe.drop(['enfermo'],1))


array([[70.,  1.,  4., ...,  2.,  3.,  3.],
       [67.,  0.,  3., ...,  2.,  0.,  7.],
       [57.,  1.,  2., ...,  1.,  0.,  7.],
       ...,
       [56.,  0.,  2., ...,  2.,  0.,  3.],
       [57.,  1.,  4., ...,  2.,  0.,  6.],
       [67.,  1.,  4., ...,  2.,  3.,  3.]])

4. Cree un modelo de regresión logística y entrénalo.

In [59]:
from sklearn.model_selection import train_test_split

#Separo los datos de "train" en entrenamiento y prueba para probar los algoritmos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

In [60]:
from sklearn.preprocessing import StandardScaler
escalar = StandardScaler()

In [61]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [56]:
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [57]:
model.score(X,y)

0.8518518518518519

5. Finalmente Use la función F de Spark SQL para crear una nueva columna correcta cuando ENFERMO() es igual a la predicción, lo que significa que el resultado predicho es igual a los resultados reales.

In [73]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
len(y_pred)

135

In [79]:
y_pred 

array([0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 0], dtype=int64)

In [88]:
y_real1 = data['enfermo']
y_real = y_real1[:135].to_numpy()
len(y_real)

135

In [89]:
y_real

array([1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 1, 0], dtype=int64)

In [90]:
print(y_real == y_pred)

[False  True  True False  True False  True  True False False  True  True
  True False False  True  True  True  True False False  True False False
  True  True False False False  True False False False  True  True False
 False False  True  True False False False  True  True False False  True
 False  True  True  True  True False  True  True  True  True False  True
 False  True  True  True  True  True False False  True False False  True
  True False  True False False False False False False  True False  True
 False  True False  True False  True False  True False  True False  True
  True False  True  True  True False False  True False False  True False
  True  True  True  True  True  True  True  True False False False False
  True  True  True  True  True  True  True False  True False False False
  True  True  True]
